In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from elm_data.step_5_label_elms.elm_labeling_app import BES_ELM_Labeling_App

In [2]:
plt.close('all')
widgets.Widget.close_all()

app = BES_ELM_Labeling_App(
    data_hdf5_file='/home/smithdr/ml/elm_data/step_4_shot_partial_data/data_v4.hdf5',
    elm_labels_hdf5_file='./labeled_elms_v2.hdf5',
    truncate=False,
)

Contents of /home/smithdr/ml/elm_data/step_4_shot_partial_data/data_v4.hdf5
Group / with 888 elements
Group /162303 with 16 elements
  Dataset FS03: (113341,) float32
  Dataset FS04: (113341,) float32
Skipping additional items...
Group /162307 with 16 elements
  Dataset FS03: (113341,) float32
  Dataset FS04: (113341,) float32
Skipping additional items...
Skipping additional items...
Contents of /fusion/projects/diagnostics/bes/smithdr/edgeml/elm_data/step_5_label_elms/labeled_elms_v2.hdf5
Group / with 2 elements
Group /elms with 59 elements
Group /elms/000000 with 0 elements
Group /elms/000001 with 0 elements
Group /elms/000002 with 0 elements
Skipping additional items...
Group /shots with 7 elements
Group /shots/166434 with 16 elements
  Dataset FS03: (251201,) float32
  Dataset FS04: (251201,) float32
  Dataset FS05: (251201,) float32
Skipping additional items...
Group /shots/166615 with 16 elements
  Dataset FS03: (257126,) float32
  Dataset FS04: (257126,) float32
  Dataset FS05: 

In [ ]:
widgets.AppLayout(
    left_sidebar=app.controls,
    center=app.fig.canvas,
    footer=app.status_label,
    pane_widths=['160px', 1, 0],
    pane_heights=[0, 1, '40px'],
)